In [2]:
import pandas as pd
import numpy as np
import statistics
import math
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.svm import SVR


In [3]:
listings_df = pd.read_csv('listings_new.csv')

#Remove unwanted characters from the column
listings_df['host_response_rate'] = listings_df['host_response_rate'].astype('str')
for i, v in listings_df['host_response_rate'].items():
    if v != 'nan':
        listings_df.at[i, 'host_response_rate'] = int(v.replace('%', ''))/100
    else:
        listings_df.at[i, 'host_response_rate'] = None

#Drop NA from these columns
listings_df.dropna(subset=['host_response_rate', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication','positivity_mean','negativity_mean','positivity_simple_mean','negativity_simple_mean'], inplace = True)

#Remove unwanted characters from price column
for i in listings_df.index:
    listings_df['price'][i] = listings_df['price'][i].replace('$', '')
    listings_df['price'][i] = listings_df['price'][i].replace(' ', '')
    listings_df['price'][i] = listings_df['price'][i].replace(',', '')

/Users/kairuoyan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/Users/kairuoyan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/kairuoyan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [4]:
#Question 6:
#train/test split CV

#Create x and y for the model
X = listings_df[['host_response_rate', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication','positivity_mean','negativity_mean','positivity_simple_mean','negativity_simple_mean']]
y = listings_df['price']

#Split that dataset into 30% test set and 70% train set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

#Create the linear model
lm = LinearRegression()
reg = lm.fit(X_train, y_train)

#Model's coefficient and intercept
model1_coef = reg.coef_
model1_intercept = lm.intercept_

print(model1_coef)

[-2.59893062e+01  1.56956603e+00 -1.63870854e+01  1.77578109e+01
 -1.34147915e+01 -2.58850663e+00  4.88852924e+01  2.66385521e+02
 -2.11877862e+03 -6.26859630e+02]


In [5]:
#K-fold CV (where k=5)

#Split the dataset into 5
kfold = KFold(n_splits=5, random_state = 42, shuffle = False)
KFold_coef_list, KFold_intercept_list = [], []
#Train the model
for train_index, test_index in kfold.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    lm = LinearRegression()
    reg = lm.fit(X_train, y_train)
    KFold_coef_list.append(list(reg.coef_))
    KFold_intercept_list.append(reg.intercept_)

#Calculating coefficient and intercept
model2_intercept = statistics.median(KFold_intercept_list) 
model2_coef = []
for i in range(len(KFold_coef_list[0])):
    model2_coef.append(statistics.median([item[i] for item in KFold_coef_list]))
    
print(model2_coef)


[-4.007116527223927, 1.4907603337907385, -13.47191162325123, 17.05087383942424, -13.351126483292495, -6.470245663104502, 71.08305110846904, 22.19893457219517, -1977.5292168665121, 1811.788751219187]


/Users/kairuoyan/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [6]:
#LOOCV

#Split the dataset in n-times (n = number of observation in the dataset)
kf = KFold(n_splits=len(listings_df), random_state = 103, shuffle = False)
KFold_coef_list, KFold_intercept_list = [], []

#print(kf)
#Train the model
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    lm = LinearRegression()
    reg = lm.fit(X_train, y_train)
    KFold_coef_list.append(list(reg.coef_))
    KFold_intercept_list.append(reg.intercept_)

#Calculating model's coefficient and intercept
model3_intercept = statistics.median(KFold_intercept_list)
model3_coef = []
for i in range(len(KFold_coef_list[0])):
    model3_coef.append(statistics.median([item[i] for item in KFold_coef_list]))
    


/Users/kairuoyan/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [7]:
#Question 7:
def squared_error(y_observed,y_predicted):
    y_observed = np.array(y_observed, dtype = np.float64)
    y_predicted = np.array(y_predicted, dtype = np.float64)
    return sum((y_predicted - y_observed) * (y_predicted - y_observed))

def r_squared(y_observed,y_predicted):
    y_observed = np.array(y_observed, dtype = np.float64)
    y_predicted = np.array(y_predicted, dtype = np.float64)
    y_mean = [y_observed.mean() for y in y_observed]
    squared_error_regr = squared_error(y_observed, y_predicted)
    squared_error_mean = squared_error(y_observed, y_mean)
    return 1 - (squared_error_regr/squared_error_mean) 

def mean_squared_error(y_observed,y_predicted):
    y_observed = np.array(y_observed, dtype = np.float64)
    y_predicted = np.array(y_predicted, dtype = np.float64)
    return (squared_error(y_observed,y_predicted))*(1/len(y_observed))

def root_mean_squared_error(y_observed, y_predicted):
    y_observed = np.array(y_observed, dtype = np.float64)
    y_predicted = np.array(y_predicted, dtype = np.float64)
    return math.sqrt(mean_squared_error(y_observed,y_predicted))

def MAPE(y_observed, y_predicted):
    y_observed = np.array(y_observed, dtype = np.float64)
    y_predicted = np.array(y_predicted, dtype = np.float64)
    return (abs((y_observed - y_predicted)/y_observed) * 100).mean()

In [14]:
#Build a function to predict y
def predict(intercept, list_coef, X):
    y_predicted = []
    for index, row in X.iterrows():
        y = intercept
        for i in range(len(row)):
            y += list_coef[i] * row[i]
        y_predicted.append(y)
    return y_predicted

#y values predicted for each model
y_predicted1 = predict(model1_intercept, model1_coef, X)
y_predicted2 = predict(model2_intercept, model2_coef, X)
y_predicted3 = predict(model3_intercept, model3_coef, X)
#print(y_predicted1)

In [8]:
#Create a table that shows different error rates corresponding to different types of cross-validations and cost functions
d = {'CV': ['Train/Test CV', 'K-Fold CV', 'LOOCV'], 'R-squared': [round(r_squared(y, y_predicted1),3), round(r_squared(y, y_predicted2),3), round(r_squared(y, y_predicted3),3)], 'MSE': [round(mean_squared_error(y, y_predicted1), 2), round(mean_squared_error(y, y_predicted2), 2), round(mean_squared_error(y, y_predicted3), 2)], 'Root MSE': [round(root_mean_squared_error(y, y_predicted1), 2), round(root_mean_squared_error(y, y_predicted2), 2), round(root_mean_squared_error(y, y_predicted3), 2)], 'MAPE': [round(MAPE(y, y_predicted1),2), round(MAPE(y, y_predicted2),2), round(MAPE(y, y_predicted3),2)]}

table = pd.DataFrame(d)

#Store the table to CSV file for later use
table.to_csv('error_table.csv', index=False)